# Credit card lead prediction

Happy Customer Bank is a mid-sized private bank that deals in all kinds of banking products, like Savings accounts, Current accounts, investment products, credit products, among other offerings.



The bank also cross-sells products to its existing customers and to do so they use different kinds of communication like tele-calling, e-mails, recommendations on net banking, mobile banking, etc. 



In this case, the Happy Customer Bank wants to cross sell its credit cards to its existing customers. The bank has identified a set of customers that are eligible for taking these credit cards.



Now, the bank is looking for your help in identifying customers that could show higher intent towards a recommended credit card, given:

Customer details (gender, age, region etc.)
Details of his/her relationship with the bank (Channel_Code,Vintage, 'Avg_Asset_Value etc.)

Train Data

#Data Dictionary= 

Variable   Definition

ID                         Unique Identifier for a row

Gender                    Gender of the Customer

Age                       Age of the Customer (in Years)

Region_Code               Code of the Region for the customers

Occupation                Occupation Type for the customer

Channel_Code              Acquisition Channel Code for the Customer  (Encoded)

Vintag                    Vintage for the Customer (In Months)

Credit_Product             If the Customer has any active credit product (Home loan,Personal loan, Credit Card etc.      

Avg_Account_Balance        Average Account Balance for the Customer in last 12 Months                                                                  


Is_Active                  If the Customer is Active in last 3 Months

Is_Lead(Target)            If the Customer is interested for the Credit Card

                            0 : Customer is not interested

                            1 : Customer is interested

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import sklearn
from pandas import DataFrame
from pandas.plotting import scatter_matrix
from pandas_profiling import ProfileReport
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import math
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
 
from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
 
from sklearn import metrics

In [ ]:
loan_train = pd.read_csv("E:\\python\\AV_hackathon\\train_s3TEQDk.csv")
loan_test = pd.read_csv("E:\\python\\AV_hackathon\\test_mSzZ8RL.csv")

Data Preprocessing

In [ ]:
loan_train.head()

In [ ]:
loan_train.tail()

In [ ]:
loan_train.columns

In [ ]:
loan_train['Age'].unique()

In [ ]:
loan_train['Age'].mean()

In [ ]:
loan_train['Age'].mode()

In [ ]:
loan_train['Occupation'].unique()

In [ ]:
loan_train['Gender'].unique()

In [ ]:
loan_train['Region_Code'].unique()

In [ ]:
loan_train['Channel_Code'].unique()

In [ ]:
loan_train['Vintage'].unique()

In [ ]:
loan_train['Credit_Product'].unique()

In [ ]:
loan_train['Avg_Account_Balance'].unique()

In [ ]:
loan_train['Avg_Account_Balance'].mean()

In [ ]:
loan_train['Avg_Account_Balance'].sum()

In [ ]:
loan_train['Avg_Account_Balance'].mode()

In [ ]:
loan_train['Is_Active'].unique()

In [ ]:
loan_train['Is_Lead'].unique()

In [ ]:
loan_train['Is_Lead'].value_counts()

In [ ]:
loan_train.dtypes

In [ ]:
loan_train.isnull()

In [ ]:
loan_train.isnull().sum()

In [ ]:
loan_train.info()

In [ ]:
loan_train.describe()

In [ ]:
loan_train.nunique()

In [ ]:
loan_train.mean()

In [ ]:
loan_train.mode()

In [ ]:
loan_train.shape

In [ ]:
print("Columns: ", len(loan_train.columns))

In [ ]:
print("Rows: ", len(loan_train))

In [ ]:
def explore_object_type(df ,feature_name):
    if df[feature_name].dtype ==  'object':
        print(df[feature_name].value_counts())

In [ ]:
explore_object_type(loan_train, 'Gender')

In [ ]:
for featureName in loan_train.columns:
    if loan_train[featureName].dtype == 'object':
        print('\n"' + str(featureName) + '\'s" Values with count are :')
        explore_object_type(loan_train, str(featureName))

In [ ]:
loan_train[loan_train["Age"] >= 75]

In [ ]:
loan_train[loan_train["Age"] >= 43]

In [ ]:
loan_train[loan_train["Age"] <= 43]

In [ ]:
loan_train.groupby('Occupation').sum()['Avg_Account_Balance']

In [ ]:
loan_train.groupby('ID').sum()['Vintage']

EDA

In [ ]:
loan_train.groupby('Occupation').sum()['Avg_Account_Balance'].plot.bar()
plt.xticks(rotation = 0)
plt.tight_layout()
plt.show()

In [ ]:
sns.boxplot(x = 'Occupation', y = 'Avg_Account_Balance', data = loan_train)

In [ ]:
loan_train.groupby('Gender').sum()['Avg_Account_Balance'].plot.bar()
plt.xticks(rotation = 0)
plt.tight_layout()
plt.show()

In [ ]:
sns.boxplot(x = 'Gender', y = 'Avg_Account_Balance', data = loan_train)

In [ ]:
loan_train.groupby('Gender').count()['Avg_Account_Balance']

In [ ]:
loan_train.groupby('Vintage').sum()['Avg_Account_Balance']

In [ ]:
loan_train.groupby('Vintage').count()['Avg_Account_Balance']

In [ ]:
sns.boxplot(x = 'Vintage', y = 'Avg_Account_Balance', data = loan_train)

In [ ]:
loan_train.groupby('Channel_Code').count()['Vintage']

In [ ]:
sns.boxplot(x = 'Channel_Code', y = 'Vintage', data = loan_train)

In [ ]:
loan_train.groupby('Channel_Code').count()['Avg_Account_Balance']

In [ ]:
sns.boxplot(x = 'Channel_Code', y = 'Avg_Account_Balance', data = loan_train)

In [ ]:
sns.boxplot(x = 'Credit_Product', y = 'Avg_Account_Balance', data = loan_train)

In [ ]:
sns.boxplot(x = 'Gender', y = 'Avg_Account_Balance', data = loan_train)

In [ ]:
sns.boxplot(x = 'Credit_Product', y = 'Vintage', data = loan_train)

In [ ]:
sns.catplot(y='Avg_Account_Balance',x='Is_Active', kind= 'point', data=loan_train, color='Red')
plt.figure(figsize=(8,10)) 

In [ ]:
sns.catplot(x='Age',y='Is_Active', kind= 'point', data=loan_train, color='Red')
plt.figure(figsize=(8,10)) 

In [ ]:
sns.countplot(x="Region_Code", data=loan_train)
print( "Total nos of region_code are ", loan_train["Region_Code"].nunique())
plt.figure(figsize = (20,8))

In [ ]:
sns.distplot(loan_train["Age"],bins=30)

In [ ]:
sns.boxplot(loan_train["Age"])

correlation between variables

In [ ]:
corelation= loan_train.corr()

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
sns.heatmap(corelation, xticklabels = corelation.columns, yticklabels = corelation.columns, annot = True)

In [ ]:
loan_train.hist()
plt.show()

In [ ]:
scatter_matrix(loan_train)
plt.figure(figsize=(8,10))
plt.show()

In [ ]:
sns.relplot(x = 'Age', y = 'Region_Code', data = loan_train)

In [ ]:
sns.relplot(x = 'Age', y = 'Avg_Account_Balance', data = loan_train)

In [ ]:
sns.relplot(x = 'Gender', y = 'Avg_Account_Balance', data = loan_train)

In [ ]:
sns.relplot(x = 'Age', y = 'Vintage', data = loan_train)

In [ ]:
sns.relplot(x = 'Credit_Product', y = 'Vintage', data = loan_train)

In [ ]:
profile = ProfileReport(loan_train, title="Creditcard_lead_predict", explorative=True)
profile

Missing Value Treatment

In [ ]:
import missingno as msno

In [ ]:
msno.bar(loan_train, color="Orange")

In [ ]:
plt.figure(figsize = (16,5),dpi = 100)
sns.heatmap(loan_train.isnull(),yticklabels=False);

In [ ]:
loan_train['Is_Lead'].value_counts()

In [ ]:
loan_train.Credit_Product.mode()

In [ ]:
fig,ax = plt.subplots(2,3,figsize=(16,10))
sns.countplot('Credit_Product',data=loan_train,ax=ax[0][0])
sns.countplot('Is_Active',data=loan_train,ax=ax[0][1])
sns.countplot('Gender',data=loan_train,ax=ax[0][2])

In [ ]:
loan_test.shape

In [ ]:
from wordcloud import WordCloud 
Active = loan_train
plt.subplots(figsize=(8,8))
wordcloud = WordCloud(
                          background_color='white',
                          width=512,
                          height=384
                         ).generate(" ".join(Active))
plt.title('Lead for Creditcard',size=25)
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

ML Modelling

In [ ]:
from sklearn.preprocessing import LabelEncoder
 
le = LabelEncoder()
 
loan_train['Gender']= le.fit_transform(loan_train['Gender'])
loan_train['Occupation']= le.fit_transform(loan_train['Occupation'])
loan_train['Credit_Product']= le.fit_transform(loan_train['Credit_Product'])
loan_train['Vintage']= le.fit_transform(loan_train['Vintage'])
loan_train['Region_Code']= le.fit_transform(loan_train['Region_Code'])
loan_train['Channel_Code']= le.fit_transform(loan_train['Channel_Code'])
loan_train['Age']= le.fit_transform(loan_train['Age'])
loan_train['Is_Lead']= le.fit_transform(loan_train['Is_Lead'])
loan_train['Is_Active']= le.fit_transform(loan_train['Is_Active'])
loan_train['ID']= le.fit_transform(loan_train['ID'])

In [ ]:
target=loan_train.iloc[:,10:]
input=loan_train.iloc[:,0:10]
target,input

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
from pylab import rcParams
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, confusion_matrix
from sklearn.metrics import f1_score, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [ ]:
pip install -U imbalanced-learn

In [ ]:
%matplotlib inline
np.random.seed(27)
rcParams['figure.figsize'] = 10, 6
warnings.filterwarnings('ignore')

In [ ]:
def generate_model_report(y_actual, y_predicted):
    print("Accuracy = " , accuracy_score(y_actual, y_predicted))
    print("Precision = " ,precision_score(y_actual, y_predicted))
    print("Recall = " ,recall_score(y_actual, y_predicted))
    print("F1 Score = " ,f1_score(y_actual, y_predicted))
    pass

In [ ]:
def generate_auc_roc_curve(clf, X_test):
    y_pred_proba = clf.predict_proba(X_test)[:, 1]
    fpr, tpr, thresholds = roc_curve(Y_test,  y_pred_proba)
    auc = roc_auc_score(Y_test, y_pred_proba)
    plt.plot(fpr,tpr,label="AUC ROC Curve with Area Under the curve ="+str(auc))
    plt.legend(loc=4)
    plt.show()
    pass

In [ ]:
target = 'Is_Lead'
df1=loan_train.copy()
X = df1.loc[:, df1.columns!=target]
Y = df1.loc[:, df1.columns==target]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, 
                                                    test_size=0.33, 
                                                    random_state=42)
print(df1[target].value_counts())

In [ ]:
import xgboost as xg
from xgboost import XGBClassifier
 
clf2 = xg.XGBClassifier(class_weight='balanced').fit(X_train, Y_train)

In [ ]:
clf2 = xg.XGBClassifier(class_weight='balanced').fit(X_train, Y_train)

In [ ]:
from sklearn.utils import class_weight
class_weight.compute_class_weight('balanced', np.unique(Y_train), Y_train[target])

In [ ]:
Y_Test_Pred = clf2.predict(X_test)

In [ ]:
generate_model_report(Y_test, Y_Test_Pred)

In [ ]:
generate_auc_roc_curve(clf2, X_test)

In [ ]:
from joblib import dump, load
dump(clf2, 'extra1.joblib')

In [ ]:

loan_test['Credit_Product']=loan_test['Credit_Product'].astype(str)
df=loan_test.copy()
lab=LabelEncoder()
df["Gender"]=lab.fit_transform(loan_test["Gender"])
df["Occupation"]=lab.fit_transform(loan_test["Occupation"])
df["Credit_Product"]=lab.fit_transform(loan_test["Credit_Product"])
df["Is_Active"]=lab.fit_transform(loan_test["Is_Active"])
df["Channel_Code"]=lab.fit_transform(loan_test["Channel_Code"])
df["ID"]=lab.fit_transform(loan_test["ID"])
df["Age"]=lab.fit_transform(loan_test["Age"])
df["Region_Code"]=lab.fit_transform(loan_test["Region_Code"])
df["Vintage"]=lab.fit_transform(loan_test["Vintage"])
df["Avg_Account_Balance"]=lab.fit_transform(loan_test["Avg_Account_Balance"])
df.head()
# target1=df.iloc[:,10:]
input1=df[0:]
input1

In [ ]:
pre=load("extra1.joblib")
predicted=pre.predict(input1)
ff=pd.DataFrame({"ID":input1["ID"],
                 "Is_Lead"
                 :predicted})
ff.to_csv ('dataframe.csv', index = None, header=True)

In [ ]:
print(ff)

In [ ]:
ff.to_csv (r'E:\\Python\\AV_hackathon\\export_ff.csv', index = False, header=True)

In [ ]:
minority_class_len = len(loan_train[loan_train[target] == 1])
print(minority_class_len)

majority_class_indices = loan_train[loan_train[target] == 0].index
print(majority_class_indices)

In [ ]:
random_majority_indices = np.random.choice(majority_class_indices,
                                           minority_class_len, 
                                           replace=False)
print(len(random_majority_indices))
minority_class_indices = loan_train[loan_train[target] == 1].index
print(minority_class_indices)

In [ ]:
under_sample_indices = np.concatenate([minority_class_indices,random_majority_indices])
under_sample = loan_train.loc[under_sample_indices]

In [ ]:
from sklearn.naive_bayes import GaussianNB 

X = under_sample.loc[:, loan_train.columns!=target]
Y = under_sample.loc[:, loan_train.columns==target]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import ExtraTreeRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn import linear_model
from catboost import CatBoostRegressor
params = {'n_estimators':10000, 'learning_rate': 0.1, 'eval_metric':'RMSE', 'loss_function':'RMSE', 'random_state':42}
model = CatBoostRegressor(**params)
model.fit(X,Y,verbose=1000)
predictions_catboost = model.predict(input1)

In [ ]:
from joblib import dump, load
dump(model, 'extra2.joblib') 

In [ ]:
loan_test['Credit_Product']=loan_test['Credit_Product'].astype(str)
df2=loan_test.copy()
lab=LabelEncoder()
df2["Gender"]=lab.fit_transform(loan_test["Gender"])
df2["Occupation"]=lab.fit_transform(loan_test["Occupation"])
df2["Credit_Product"]=lab.fit_transform(loan_test["Credit_Product"])
df2["Is_Active"]=lab.fit_transform(loan_test["Is_Active"])
df2["Channel_Code"]=lab.fit_transform(loan_test["Channel_Code"])
df2["ID"]=lab.fit_transform(loan_test["ID"])
df2["Age"]=lab.fit_transform(loan_test["Age"])
df2["Region_Code"]=lab.fit_transform(loan_test["Region_Code"])
df2["Vintage"]=lab.fit_transform(loan_test["Vintage"])
df2["Avg_Account_Balance"]=lab.fit_transform(loan_test["Avg_Account_Balance"])
df2.head()

In [ ]:
input1=df2.iloc[:,0:]
input1

In [ ]:
pred1=load("extra2.joblib")
predicted=pred1.predict(input1)
ff=pd.DataFrame({"ID":loan_test["ID"],
                 "Is_Lead"
                 :predicted})

ff.to_csv("extra2.csv")

In [ ]:
ff.to_csv (r'E:\\Python\\AV_hackathon\\export_extra2.csv', index = False, header=True)

In [ ]:
X = under_sample.loc[:, loan_train.columns!=target]
Y = under_sample.loc[:, loan_train.columns==target]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
clf = ExtraTreesClassifier().fit(X_train, Y_train)
Y_Test_Pred = clf.predict(X_test)

In [ ]:
clf = ExtraTreesClassifier().fit(X_train, Y_train)
Y_Test_Pred = clf.predict(X_test)

In [ ]:
generate_model_report(Y_test, Y_Test_Pred)

In [ ]:
generate_auc_roc_curve(clf, X_test)

In [ ]:
from joblib import dump, load
dump(clf, 'extra3.joblib')

In [ ]:
pre=load("extra3.joblib")
predicted=pre.predict(input1)
ff=pd.DataFrame({"ID":loan_test["ID"],
                 "Is_Lead"
                 :predicted})

ff.to_csv("extra3.csv")

In [ ]:
ff.to_csv (r'E:\\Python\\AV_hackathon\\export_extra3.csv', index = False, header=True)